In [1]:
@time using Clp
@time using JuMP
@time using PyPlot

 18.304791 seconds (909.60 k allocations: 63.951 MiB, 0.84% gc time, 0.21% compilation time)
  0.785383 seconds (166.83 k allocations: 10.230 MiB, 18.57% gc time)
 39.611853 seconds (734.29 k allocations: 53.815 MiB, 0.24% gc time, 3.63% compilation time: 93% of which was recompilation)


In [2]:
versioninfo()

Julia Version 1.10.5
Commit 6f3fdf7b362 (2024-08-27 14:19 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 4 × Intel(R) Core(TM) i3-7020U CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)
Environment:
  JULIA_NUM_THREADS = 


## Problema 1: (Solução de sistema linear via LP e forma padrão) 

Considere o sistema de equações lineares abaixo:

$$
\begin{array}{rlr}
2x_1 - x_2 + x_3 & = & 2\\
3x_1 + 5x_2 + 5x_3 & = & 3.5 \\
x_1 + x_2 + x_3 & = & 0.5
\end{array}
$$

 - Determina a solução do sistema transformando-o em programa linear (PL), utilizando JuMP/Julia. _Dica_: Utilize o conjunto de "Transformation tricks" (slides 3-21 em diante). O desafio passa a ser a definição de uma função objetivo adequada!
 - Escreva o PL que você definiu no item anterior na forma padrão $\max {c}^\top {x}$, sujeito a ${Ax}\leq {b}, {x}\geq 0$. Resolva de novo em JuMP, utilizando a forma padrão matricial. Compare as soluções obtidas.

### Solução do Problema 1

Para formular o problema como um programa linear, primeiro se faz necessário a escolha de uma função custo adequada ao problema, dessa forma será utilizada a soma das variáveis de forma que não aja nenhum peso e simplifique a otimização, garantindo uma factibilidade maior.
 
\begin{align*}
\text{minimizar} \quad & x_1 + x_2 + x_3 \\
\text{sujeito a} \quad & 2x_1 - x_2 + x_3 = 2 \\
& 3x_1 + 5x_2 + 5x_3 = 3.5 \\
& x_1 + x_2 + x_3 = 0.5 \\
& x_1, x_2, x_3 \in \mathbb{R}
\end{align*}



In [9]:
m = Model(Clp.Optimizer)
@variable(m, x1)
@variable(m, x2)
@variable(m, x3)
@constraint(m, 2x1 - x2 + x3 == 2)
@constraint(m, 3x1 + 5x2 + 5x3 == 3.5)
@constraint(m, x1 + x2 + x3 == 0.5)
@objective(m, Min, x1 + x2 + x3)

optimize!(m)

println(m)
println()
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))

Min x1 + 8 x2 + x3
Subject to
 2 x1 - x2 + x3 = 2
 3 x1 + 5 x2 + 5 x3 = 3.5
 x1 + x2 + x3 = 0.5


x1 = -0.5
x2 = -1.0
x3 = 2.0
Coin0506I Presolve 0 (-3) rows, 0 (-3) columns and 0 (-9) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value -6.5
Coin0511I After Postsolve, objective -6.5, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -6.5 - 0 iterations time 0.002, Presolve 0.00


### Forma Padrão de Programação Linear

Colocando o programa linear na forma padrão temos que escrever na forma matricial e limitar $$\mathbf{A} \mathbf{x} \leq \mathbf{b}$$
Dessa forma tem-se a seguinte representação:



maximizar:

$$
\mathbf{c}^\top \mathbf{x} = \begin{bmatrix} 1 & 1 & 1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}
$$

sujeito a:

$$
\begin{bmatrix} 
2 & -1 & 1 \\ 
3 & 5 & 5 \\ 
1 & 1 & 1 
\end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} \leq \begin{bmatrix} 2 \\ 3.5 \\ 0.5 \end{bmatrix}
$$
$$
x_1, x_2, x_3 \geq 0
$$


In [11]:
m = Model(Clp.Optimizer)
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)
@variable(m, x3 >= 0)
@constraint(m, 2x1 - x2 + x3 <= 2)
@constraint(m, 3x1 + 5x2 + 5x3 <= 3.5)
@constraint(m, x1 + x2 + x3 <= 0.5)
@objective(m, Max, x1 + x2 + x3)

optimize!(m)

println(m)
println()
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println("x3 = ", value(x3))

Max x1 + x2 + x3
Subject to
 2 x1 - x2 + x3 ≤ 2
 3 x1 + 5 x2 + 5 x3 ≤ 3.5
 x1 + x2 + x3 ≤ 0.5
 x1 ≥ 0
 x2 ≥ 0
 x3 ≥ 0


x1 = 0.0
x2 = 0.5
x3 = 0.0
Coin0506I Presolve 2 (-1) rows, 2 (-1) columns and 4 (-5) elements
Clp0006I 0  Obj -0 Dual inf 1.999998 (2)
Clp0006I 2  Obj 0.5
Clp0000I Optimal - objective value 0.5
Coin0511I After Postsolve, objective 0.5, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.5 - 2 iterations time 0.002, Presolve 0.00


Os resultados se diferem pelo fato de que na programação linear tem-se primeiramente uma minimização e sem nenhuma restrição de negatividade, quanto que com a forma padrão foi feita uma maximização na função custo com as restrições de $$ x \geq 0$$ e $$\mathbf{A} \mathbf{x} \leq \mathbf{b}$$ 